In [2]:
import face_recognition
import cv2

In [3]:
import os
import glob
import numpy as np

In [3]:
img = cv2.imread("images/1.jpg")
rgb = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img_en = face_recognition.face_encodings(rgb)[0]

In [5]:
img2 = cv2.imread("2.jpg")
rgb2 = cv2.cvtColor(img2,cv2.COLOR_BGR2RGB)
img_en2 = face_recognition.face_encodings(rgb2)[0]

In [6]:
print(face_recognition.compare_faces([img_en],img_en2))

[True]


In [4]:
known_face_encodings=[]
known_face_name =[]

In [5]:
def load_encoding_images(imagesPath):
    imagesPath = glob.glob(os.path.join(imagesPath,"*.*"))
    print("{} Encodings found".format(len(imagesPath)))
    for image in imagesPath:
        img = cv2.imread(image)
        rgb_img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        
        (filename,ext)=os.path.splitext(os.path.basename(image))
        
        img_enco = face_recognition.face_encodings(rgb_img)[0]
        
        known_face_encodings.append(img_enco)
        known_face_name.append(filename)
    print("Encoding images loaded")

In [6]:
def detect_known_faces(frame):
    sframe = cv2.resize(frame,(0,0),fx=0.25,fy=0.25)
    srgb =cv2.cvtColor(sframe,cv2.COLOR_BGR2RGB)
    face_loc = face_recognition.face_locations(srgb)
    face_en = face_recognition.face_encodings(srgb,face_loc)
    
    face_names =[]
    for face_enco in face_en:
        matches = face_recognition.compare_faces(known_face_encodings,face_enco)
        name = "Unknown"
        
        face_dist = face_recognition.face_distance(known_face_encodings,face_enco)
        best_index = np.argmin(face_dist)
        if matches[best_index]:
            name = known_face_name[best_index]
        face_names.append(name)
    face_loc = np.array(face_loc)
    face_loc = face_loc/0.25
    return face_loc.astype(int),face_names

In [7]:
load_encoding_images("images/")


1 Encodings found
Encoding images loaded


In [8]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret,frame = cap.read()
    face_loc,face_names = detect_known_faces(frame)
    for loc,name in zip(face_loc,face_names):
        y1,x2,y2,x1 = loc[0],loc[1],loc[2],loc[3]
        
        cv2.putText(frame,name,(x1,y1-10),cv2.FONT_HERSHEY_DUPLEX,1,(0,255,0),2)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),4)
        
    cv2.imshow("Frame",frame)
    
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

In [6]:
import face_recognition
import cv2
import os
import glob
import numpy as np

class SimpleFacerec:
    def __init__(self):
        self.known_face_encodings = []
        self.known_face_names = []

        # Resize frame for a faster speed
        self.frame_resizing = 0.25

    def load_encoding_images(self, images_path):
        """
        Load encoding images from path
        :param images_path:
        :return:
        """
        # Load Images
        images_path = glob.glob(os.path.join(images_path, "*.*"))

        print("{} encoding images found.".format(len(images_path)))

        # Store image encoding and names
        for img_path in images_path:
            img = cv2.imread(img_path)
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Get the filename only from the initial file path.
            basename = os.path.basename(img_path)
            (filename, ext) = os.path.splitext(basename)
            # Get encoding
            img_encoding = face_recognition.face_encodings(rgb_img)[0]

            # Store file name and file encoding
            self.known_face_encodings.append(img_encoding)
            self.known_face_names.append(filename)
        print("Encoding images loaded")

    def detect_known_faces(self, frame):
        small_frame = cv2.resize(frame, (0, 0), fx=self.frame_resizing, fy=self.frame_resizing)
        # Find all the faces and face encodings in the current frame of video
        # Convert the image from BGR color (which OpenCV uses) to RGB color (which face_recognition uses)
        rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # See if the face is a match for the known face(s)
            matches = face_recognition.compare_faces(self.known_face_encodings, face_encoding)
            name = "Unknown"

            # # If a match was found in known_face_encodings, just use the first one.
            # if True in matches:
            #     first_match_index = matches.index(True)
            #     name = known_face_names[first_match_index]

            # Or instead, use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = self.known_face_names[best_match_index]
            face_names.append(name)

        # Convert to numpy array to adjust coordinates with frame resizing quickly
        face_locations = np.array(face_locations)
        face_locations = face_locations / self.frame_resizing
        return face_locations.astype(int), face_names


In [7]:
import cv2

# Encode faces from a folder
sfr = SimpleFacerec()
sfr.load_encoding_images("images/")

# Load Camera
cap = cv2.VideoCapture(0)


while True:
    ret, frame = cap.read()

    # Detect Faces
    face_locations, face_names = sfr.detect_known_faces(frame)
    for face_loc, name in zip(face_locations, face_names):
        y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]

        cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)

    cv2.imshow("Frame", frame)

    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()

1 encoding images found.
Encoding images loaded
